so this is the followup CSVParse in its current iteration as of June 14, 2022. As of now,  CSVParse jupyter notebook functions as the initial .csv parsing, which puts every row as a dictionary into a list. as a check, items on the list are put into variables and printed onto a text file which is used as a "check." finally, base .int and .var files are created in a for loop, with one of each per molecule. the numbers of each file are associated with the key "list_value" in the original .csv.

from here, I need to modify parts of the .int and .var files to the correct, ML-predicted rotational constants and dipole moments and run it. 

instead of attempting to just get dictionaries, I'm just gonna repeat the command initially lol. at some point i'm going to want to be able to automate so that once last code is done, i do this. later hannah's problem tho


The point of this is to take the spcat files made in the CSVParse script and modify each so that it contains the values from the summary.csv files. 

# comments as of uploading:
this file has a few roles. It parses your summary file, and inputs the information (ML generated and quantum calc generated) into individual .int and .var files. We then run spcat on it at 300K, with dummy variables. Running it once allows it to get the more accurate partition functions, which this file then goes through the .int/.var files to replace, and then runs spcat again. finally, that spcat is "correct" and it creates a qpart file from the results. 

those are used later in molsim.

In [ ]:
import csv
import os

get to your .csv file! recall that this should be the full ML results, including quantum chem calcs. 

In [ ]:
 
# opening the CSV file
with open('path_to_csv.csv', mode ='r') as file: 
    csvFile=csv.DictReader(file)
    
    listy = list(csvFile) 
    

IMPORTANT NOTE: if you begin to see errors such as "index out of range" this is likely an issue in the original .csv file. print(listy) if you need to check

### redefining variables

In [ ]:
#this should be replaced with  one loop in cell below;check
formula =[]
for i in listy:
    formy=i['Formula']
    formula.append(formy)
#print(formula)

smilesstring = []
for i in listy:
    smiley = i['SMILES']
    smilesstring.append(smiley)
#print(smilesstring)

rota =[]
for i in listy:
    rotya=i['A']
    rota.append(rotya)
# print(rota)

rotb =[]
for i in listy:
    rotyb=i['B']
    rotb.append(rotyb)
# print(rotb)

rotc =[]
for i in listy:
    rotyc=i['C']
    rotc.append(rotyc)
# print(rotc)

mua = []
for j in listy:
    muay = j['mu_a']
    mua.append(muay)
#print(mua)

mub = []
for k in listy:
    muby = k['mu_b']
    mub.append(muby)
#print(mub)

muc = []
for i in listy:
    mucy = i['mu_c']
    muc.append(mucy)
    
intSmile=[]
for i in listy:
    intsmiley=i['SMILES']
    intSmile.append(intsmiley)
print(intSmile)

# abInitCalc  =[]
# for i in listy:
#     abi = i['not_opt']
#     abInitCalc.append(abi)

colDens = []
for i in listy:
    cd = i['log_column']
    colDens.append(cd)
#print(colDens)

In [ ]:
#creation of all relevant lists
formula =[]
smilesstring = []
rota =[]
rotb =[]
rotc =[]
mua = []
mub = []
muc = []
intSmile=[]
colDens = []
abInitCalc  =[]


for i in listy:
    formy=i['Formula']
    formula.append(formy)
    smiley = i['SMILES']
    smilesstring.append(smiley)
    rotya=i['A']
    rota.append(rotya)
    rotyb=i['B']
    rotb.append(rotyb)
    rotyc=i['C']
    rotc.append(rotyc)
    muay = i['mu_a']
    mua.append(muay)
    muby = i['mu_b']
    mub.append(muby)
    mucy = i['mu_c']
    muc.append(mucy)
    intsmiley=i['SMILES']
    intSmile.append(intsmiley)
    cd = i['log_column']
    colDens.append(cd)
#     abi = i['not_opt']
#     abInitCalc.append(abi)
    
    
#print(formula, intSmile)
    

so the next few cells are modifying the values in our lists from the .csv file. they include filling in blanks with 0, and modifying the rotational constants into scientific notation that spcat can read.

In [ ]:
#this replaces any blanks in the .csv files with 0 for any missing rotational constant values. 
rota_replaceblanks = ['0' if x=='' else x for x in rota]

In [ ]:
newrotA = [float(x) for x in rota_replaceblanks]

now, i have a list of floats, and i want to change them into sci notation

In [ ]:
from decimal import Decimal
finalrotA = []
for i in newrotA:
    forma = '%.2E' % Decimal(i)
    finalrotA.append(forma)
    #print(forma[0])
print(finalrotA[15])
#print(type(finalrotA[20]))#mysteroiusly it is a string again?

2.91E+03


for rotb

In [ ]:
rotb_replaceblanks = ['0' if x=='' else x for x in rotb] #seems alright
#print(rotb_replaceblanks)
newrotB = [float(x) for x in rotb_replaceblanks]
#print(newrotB)#also seemso k
finalrotB = []
from decimal import Decimal
for j in newrotB:
    formb = '%.2E' % Decimal(j)
    finalrotB.append(formb)


now... for c

In [ ]:
rotc_replaceblanks = ['0' if x=='' else x for x in rotc] 
newrotC = [float(x) for x in rotc_replaceblanks]
#print(newrotB)#also seemso k
finalrotC = []
from decimal import Decimal
for k in newrotC:
    formc = '%.2E' % Decimal(k)
    finalrotC.append(formc)
#print(finalrotC)


# everything in this next few sections is to ensure you are in the right folder.
get into the right folder|
a large part of this is cause i wanted all the .int/.cat/etc files in a separate subfolder, for organization sake. 

In [ ]:
origPath = os.getcwd()

/Users/hannahts/Desktop/mcguire group/2. Observational/4. SpectralAutomation
<class 'str'>
<class 'str'>


In [ ]:
#this is the variable to change if you want to be in a diff folder. make sure it is in the current folder since 
#that is all i wrote the code for
newFolder = origPath + '/tmc1-fd-2025'

In [ ]:
os.chdir(newFolder)

In [ ]:
#this line  opens and reads a file into string.
#historically use for testing. you can skip or delete if you dont want to use it. 
with open('1.int', 'r') as myFile:
    filedata = myFile.read()
    print(filedata)

name/title
0  1  1312  00  200  -10  -10 300.0  300
 001  0.1947
 002  -3.3752
 003  1.1111



In [ ]:
#here, i try to use .replace with objects from a list
#histroically used for testing/checking
#targettitle = listy[1]['internal_smi']
targettitle = intSmile[1]
indexspot = listy.index
print(targettitle)
filedata = filedata.replace('name/title', targettitle)
print(filedata)

C/C=C/CCN
C/C=C/CCN
0  1  1312  00  200  -10  -10 300.0  300
 001  0.1947
 002  -3.3752
 003  1.1111



by comparing the above filedata with the file data just below it, you can see the success of filedata.replace. it has replaced the one area I've asked it to, i.e., the title. now, i'll likely want to for loop this to replace a few times. i don't care if its "slow", on a first pass attempt it's what i'll try

Creation of the .int files. these are done in such a way that the name of the .int file matches the place in the list of the species. this should match the original .csv file.

# editing .int

In [ ]:

folder_path = '/path/to/folder/just/created/above/tmc1-fd-2025'
length_list = 1000 #change to the number of molecules in the .csv file you are looking at

for i in range(length_list): #updated
    with open(f'{folder_path}/{i}.int', 'w') as f:
        f.write(smilesstring[i] +"           "+ " mlc list_place is  "+ str(i) + "\n"
               "0  1  1312  00  250  -10  -10 300.0  300 \n" +
               " 001  " + mua[i] + "\n"
               " 002  " + mub[i] + "\n"
               " 003  " + muc[i])
    
            
        #  (f write) title - smilesstring[i]
        #f write ('spcat values 0 1 etc')
        #f write '001'+ (mucA[i])

                    
                
                  
    


# editing .var

In [ ]:
#for i in range(1510):
for i in range(1925):
    with open(f'{folder_path}/{i}.var', 'w') as f:
        f.write(smilesstring[i] +"           "+ "mlc list_place is "+ str(i)+ " NOT \n"
               "   8   33   40    0     0.0000E+00     5.0000E+20     1.0000E+00 1.0000000000 \n" +
                "a   1  1  0  50  0  1  1  1  1  -1   0 \n"
               "        10000   " + finalrotA[i]  + " 1.00000000E-37 /A \n"
               "        20000   " + finalrotB[i] + " 1.00000000E-37 /B \n"
               "        30000   " + finalrotC[i] + " 1.00000000E-37 /C \n")
       
            

# Running spcat
this uses the OS terminal and is not done on jupyter notebook itself. As such, you have to keep an eye on the terminal window you opened this notebook with. Even if the jupyter notebook says its done, the OS may still be running. The easiest way to check is to attempt to print like, the length of something like the partFn and see what it does. if it gives you a value, the it's done; if not, it's still running spcat in the background

In [ ]:

import subprocess

In [ ]:
for i in range(length_list):
    subprocess.Popen(["spcat", f"{i}.int", f"{i}.var"])

# IMPORTANT NOTE!!
sometimes, the jupyter notebook will say it's done running spcat before spcat has officially run through all the files. if you open jupyter through terminal (as I recommend), the terminal may show ongoing calculations. WAIT U NTIL THOSE ARE DONE or your results will be bad!

## creation of the filecount.sh file and the spcat check flag system file. you will need to ensure you have this file written. i'll have to add that as a code in later. 
this is a .sh file, totally optional step. 

In [ ]:
#this prints the number of files to a text, as a sort of check if you want it. below, i run the command directly
#in jupyter so you can see it. you should get 1509, which in this case, is the number of molecules. 
import os
os.popen('sh filecount.sh')

In [ ]:
import subprocess
#expect number equal to len(partFn). or double, if you're running this AFTER the rest of the script which creates a new *partFn.out


command = "ls -1 *.out | wc -l"
result = subprocess.run(command, shell=True, text=True, capture_output=True)

print(f"Number of .out files: {result.stdout.strip()}")


Number of .out files: 1925


### Flag systems below

below calls a file that will check and sort the length of the various .out files to a text file called filelengths.txt. you can easily check to make sure all the files are the same length which will suggest that the next step to pull proper Q is alright as well. 

In [ ]:
import os
os.popen('sh spcatcheck.sh')



## Parsing .out Files for calculated partition function Q at 300K

In [ ]:

folder_path = '/path/to/folder/made/above/tmc1-fd-2025'



partFn = []

for i in range(length_list):
    with open(f'{folder_path}/{i}.out', 'r') as rd:
        for line in rd:
            if line =="TEMPERATURE - Q(SPIN-ROT.) - log Q(SPIN-ROT.)\n":
                break

        for line in rd:
            temp, Q, logQ = line.split()
            if float(temp) == 300:
                partFn.append(Q)
                break
          #  partFn.append(Q)
#print(Q)
        
        
        
        


## the cell below replaces the original .int files initial partition function value of 1312 with the calculated one from the list above, partFn

In [ ]:
for i in range(1925):
    with open(f'{folder_path}/{i}.var', 'w') as f:
        f.write(smilesstring[i] +"           "+ "mlc list_place is "+ str(i)+ "\n"
               "   8   33   40    0     0.0000E+00     5.0000E+20     1.0000E+00 1.0000000000 \n" +
                "a   1  1  0  50  0  1  1  1  1  -1   0 \n"
               "        10000   " + finalrotA[i]  + " 1.00000000E-37 /A \n"
               "        20000   " + finalrotB[i] + " 1.00000000E-37 /B \n"
               "        30000   " + finalrotC[i] + " 1.00000000E-37 /C \n")

In [ ]:


input_directory = folder_path


        
def replace_value_in_file(old_file_path, new_file_path, old_value, new_value):
    with open(old_file_path, 'r') as file:
        content = file.read()
    
    # Replace the old value with the new value
    new_content = content.replace(old_value, new_value)
    
    # Write the modified content to the new file
    with open(new_file_path, 'w') as file:
        file.write(new_content)
    
        

# Loop through each number in the specified range
for i in range(0,1925):
    # Construct the original filename and new filename
    original_file_name = f"{i}.int"  
    original_file_path = os.path.join(input_directory, original_file_name)
    new_file_name = f"{i}partFn.int"
    new_file_path = os.path.join(input_directory, new_file_name)
    
    # Ensure the file exists
    if os.path.exists(original_file_path):
        replace_value_in_file(original_file_path, new_file_path, '0  1  1312  00  250  -10  -10 300.0  300', '0  1  '+partFn[i]+'  00  250  -10  -10 300.0  300')
    else:
        print(f"File {original_file_path} does not exist.")

print("Replacement done!")
        
        
        
        
        
        
        
        
        


Replacement done!


# Rerun spcat. now it has to correct .int and .var with correct tempts

In [ ]:
#running spcat again
#important to note that spcat names the result files based on .int name
for i in range(length_list):
    subprocess.Popen(["spcat", f"{i}partFn.int", f"{i}.var"])

# Creation of qpart files


finally, to use molsim, we use the qpart files. these just have the relevant temperature calcs. 

In [ ]:
#creates qpart files



folder_path = '/path/to/above/tmc1-fd-2025' #i redefine this multiple times because i'm paranoid, not for any other particular reason



listOfDicts=[]


for i in range(length_list):
    with open(f'{folder_path}/{i}partFn.out', 'r') as rd:
        for line in rd:
            if line =="TEMPERATURE - Q(SPIN-ROT.) - log Q(SPIN-ROT.)\n":
                break
                

        tempList = [] 
        qList = []
        for line in rd:
            temp, Q, logQ = line.split()
            tempList.append(temp)
            qList.append(Q)
            #print(type(tempList[0]))
            #print(type(tempList))
        #print(qList[1])
        #print(qList)
            
            
        with open(f'{folder_path}/{i}.qpart', 'w') as f: 
            f.write("#form : interpolation \n")
            for k in range(len(tempList)):
                f.write(tempList[k]+"   " + qList[k]+"\n")
            

            
            
            